In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie

import NamedTrajectories as NT

In [ ]:
traj = rand(NT.NamedTrajectory, 10);

In [ ]:
function convert_arguments(
    P::Type{<:Makie.Series}, 
    traj::NT.NamedTrajectory,
    name::Symbol;
    output::Union{Function, Nothing}=nothing
)
    if !isnothing(output)
        output_data = try
            stack(output.(eachcol(traj[name])))
        catch
            throw(ArgumentError("Output transformation of $(name) failed."))
        end
    else
        output_data = traj[name]
    end

    return convert_arguments(P, NT.get_times(traj), output_data)
end

MakieCore.used_attributes(::Type{<:Makie.Series}, ::NT.NamedTrajectory, ::Symbol) = (:output,)
MakieCore.plottype(::NT.NamedTrajectory, ::Symbol) = Makie.Series

In [ ]:
f = Figure()
ax = Axis(f[1,1])

labels = ["x$i" for i in 1:size(traj.x, 1)]

p = plot!(
    ax, traj, :x, 
    output=x -> x .^ 2, 
    labels=labels, color=:seaborn_colorblind, marker=:circle
)
Legend(f[1,2], ax)
f

In [ ]:
typeof(p)

In [ ]:
p.plots

In [ ]:
Makie.get_labeled_plots(ax, merge=false, unique=false)

I don't like recipes defined over the series because legends are awkward. I can't get the legend to be colored correctly.

In [ ]:
@recipe(NamePlot, traj, input_name, output_name, transformation) do scene
    Attributes(
        color=:seaborn_colorblind,
        linestyle=theme(scene, :linestyle),
        linewidth=theme(scene, :linewidth),
        marker=nothing,
        markersize=nothing,
    )
end

# Adds the ability to recall plot labels for a legend
Makie.get_plots(P::NamePlot) = Makie.get_plots(P.plots[1])

# plot existing component
function Makie.plot!(
    P::NamePlot{<:Tuple{<:NT.NamedTrajectory, Symbol}};
    kwargs...
)
    lift(P[:traj], P[:input_name]) do traj, name
        P = plot!(
            P, traj, name;
            labels=["$(name)$(i)" for i in eachindex(traj.components[name])],
            color = P[:color],
            linestyle = P[:linestyle],
            linewidth = P[:linewidth],
            marker = P[:marker],
            markersize = P[:markersize],
        )
        
    end
    return P
end

In [ ]:
f = Figure()
ax = Axis(f[1,1])
name = :x
p = nameplot!(ax, traj, name, linewidth=5)
Legend(f[1,2], ax)
f

In [ ]:
typeof(p)

In [ ]:
Makie.get_plots.(p.plots)

In [ ]:
Makie.get_plots(p)